# RF-DETR Training fur Tischtennis Ball & Schlager Erkennung

Dieses Notebook trainiert ein **RF-DETR** Modell (Apache 2.0 Lizenz, kommerziell nutzbar!) 
fur die Erkennung von:
- **Ball** (TT-Ball, weiss/orange, 3-10 Pixel)
- **Racket** (TT-Schlager)
- **Table** (Tischtennis-Tisch, optional)

## Voraussetzungen
1. Google Colab Account (kostenlos, T4 GPU)
2. Roboflow Account mit gelabeltem Dataset
3. Roboflow API Key

## Lizenzen (alle kommerziell nutzbar!)
- RF-DETR: Apache 2.0
- ONNX Runtime: MIT
- Supervision: MIT
- Roboflow: Dein Model gehoert dir

## Schritt 1: GPU pruefen & Pakete installieren

In [ ]:
# GPU pruefen - sollte T4 oder besser sein
!nvidia-smi

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA verfuegbar: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# RF-DETR und Abhaengigkeiten installieren
!pip install -q rfdetr roboflow supervision onnx onnxruntime

## Schritt 2: Dataset von Roboflow laden

### Vorbereitung in Roboflow:
1. Erstelle ein Projekt: "Table Tennis Detection" (Object Detection)
2. Klassen: `ball`, `racket`, `table`
3. Lade deine Videos hoch (Roboflow extrahiert Frames)
4. Zeichne Bounding Boxes um Ball, Schlager, Tisch
5. Mindestens 300+ Bilder labeln
6. Dataset Version generieren (mit Augmentation: Flip, Brightness, Blur)
7. API Key kopieren

In [ ]:
# ============================================
# HIER DEINE WERTE EINTRAGEN:
# ============================================

ROBOFLOW_API_KEY = "DEIN_API_KEY_HIER"  # Aus Roboflow Settings
ROBOFLOW_WORKSPACE = "dein-workspace"    # Dein Workspace-Name
ROBOFLOW_PROJECT = "table-tennis-detection"  # Dein Projekt-Name
ROBOFLOW_VERSION = 1  # Dataset-Version

# ============================================

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(ROBOFLOW_VERSION)

# Dataset im COCO-Format herunterladen (RF-DETR braucht COCO)
dataset = version.download("coco")

print(f"\nDataset heruntergeladen nach: {dataset.location}")
print(f"Train-Bilder: {dataset.location}/train")
print(f"Valid-Bilder: {dataset.location}/valid")
print(f"Test-Bilder:  {dataset.location}/test")

In [ ]:
# Dataset-Statistiken anzeigen
import json
import os

for split in ['train', 'valid', 'test']:
    ann_path = os.path.join(dataset.location, split, '_annotations.coco.json')
    if os.path.exists(ann_path):
        with open(ann_path) as f:
            data = json.load(f)
        cats = {c['id']: c['name'] for c in data['categories']}
        ann_counts = {}
        for ann in data['annotations']:
            name = cats[ann['category_id']]
            ann_counts[name] = ann_counts.get(name, 0) + 1
        print(f"\n{split.upper()}:")
        print(f"  Bilder: {len(data['images'])}")
        print(f"  Annotationen: {len(data['annotations'])}")
        for name, count in sorted(ann_counts.items()):
            print(f"    {name}: {count}")

## Schritt 3: RF-DETR trainieren

RF-DETR verwendet einen **DINOv2-Backbone** der besonders gut bei kleinen Objekten ist - perfekt fuer TT-Baelle!

### Training-Parameter:
- `epochs`: 50-100 (mehr = besser, aber laenger)
- `batch_size`: 8-16 (je nach GPU-Speicher)
- `imgsz`: 640 (Standard) oder 960 (besser fuer kleine Objekte, aber langsamer)
- `lr`: 1e-4 (Standard, gut fuer Fine-Tuning)

In [ ]:
from rfdetr import RFDETRBase

# RF-DETR Base Modell (guter Kompromiss aus Genauigkeit und Geschwindigkeit)
# Alternativen: RFDETRSmall (schneller), RFDETRLarge (genauer)
model = RFDETRBase()

# Training starten
model.train(
    dataset_dir=dataset.location,
    epochs=75,           # 75 Epochen - guter Kompromiss
    batch_size=8,        # 8 fuer T4 GPU (16GB VRAM)
    imgsz=640,           # 640px - Standard, fuer mehr Genauigkeit 960 nutzen
    lr=1e-4,             # Learning Rate
    grad_accum_steps=4,  # Gradient Accumulation (simuliert groessere Batch)
    use_ema=True,        # Exponential Moving Average (stabilisiert Training)
    device="cuda"
)

print("\nTraining abgeschlossen!")

## Schritt 4: Modell testen & evaluieren

In [ ]:
import supervision as sv
from PIL import Image
import glob
import matplotlib.pyplot as plt

# Test-Bilder laden
test_images = glob.glob(os.path.join(dataset.location, 'test', '*.jpg'))
test_images += glob.glob(os.path.join(dataset.location, 'test', '*.png'))

if not test_images:
    test_images = glob.glob(os.path.join(dataset.location, 'valid', '*.jpg'))
    test_images += glob.glob(os.path.join(dataset.location, 'valid', '*.png'))

print(f"{len(test_images)} Test-Bilder gefunden")

# 6 zufaellige Bilder testen
import random
sample_images = random.sample(test_images, min(6, len(test_images)))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Annotator fuer Visualisierung
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1)

for i, img_path in enumerate(sample_images):
    image = Image.open(img_path)
    detections = model.predict(image, threshold=0.3)
    
    # Supervision Detections
    import numpy as np
    annotated = np.array(image)
    annotated = box_annotator.annotate(annotated, detections)
    
    labels = []
    for j in range(len(detections)):
        conf = detections.confidence[j]
        cls_id = detections.class_id[j]
        cls_name = detections.data.get('class_name', ['?'])[j] if 'class_name' in detections.data else str(cls_id)
        labels.append(f"{cls_name} {conf:.2f}")
    
    annotated = label_annotator.annotate(annotated, detections, labels=labels)
    
    axes[i].imshow(annotated)
    axes[i].set_title(os.path.basename(img_path))
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('test_predictions.png', dpi=150)
plt.show()
print("Ergebnisse gespeichert als test_predictions.png")

## Schritt 5: ONNX Export (fuer Browser-Inferenz)

Der ONNX-Export erzeugt ein Modell das mit **onnxruntime-web** direkt im Browser laufen kann.
Das bedeutet: keine Server-Kosten, alles laeuft lokal beim User!

In [ ]:
# Bestes Modell als ONNX exportieren
model.export("tt-detector.onnx")

# Dateigröße prüfen
onnx_size = os.path.getsize('tt-detector.onnx') / (1024 * 1024)
print(f"\nONNX Modell exportiert: tt-detector.onnx")
print(f"Dateigröße: {onnx_size:.1f} MB")
print(f"\nNaechster Schritt: Lade tt-detector.onnx herunter und lege es in:")
print(f"  ttv-champions-web/public/models/tt-detector.onnx")

In [ ]:
# ONNX Modell validieren
import onnx
import onnxruntime as ort

# Modell laden und pruefen
onnx_model = onnx.load('tt-detector.onnx')
onnx.checker.check_model(onnx_model)
print("ONNX Modell ist valide!")

# Input/Output Format anzeigen
session = ort.InferenceSession('tt-detector.onnx')

print("\nInputs:")
for inp in session.get_inputs():
    print(f"  {inp.name}: {inp.shape} ({inp.type})")

print("\nOutputs:")
for out in session.get_outputs():
    print(f"  {out.name}: {out.shape} ({out.type})")

# Quick Inference Test
import numpy as np
dummy_input = np.random.randn(1, 3, 640, 640).astype(np.float32)
results = session.run(None, {session.get_inputs()[0].name: dummy_input})
print(f"\nInference Test erfolgreich!")
for i, out in enumerate(session.get_outputs()):
    print(f"  Output '{out.name}': shape={results[i].shape}")

## Schritt 6: Video-Test (optional)

Teste das Modell auf einem ganzen Video.

In [ ]:
# Optional: Video-Test
# Lade ein Test-Video hoch oder gib eine URL an

TEST_VIDEO = "test_match.mp4"  # Pfad zum Test-Video

if os.path.exists(TEST_VIDEO):
    import cv2
    
    cap = cv2.VideoCapture(TEST_VIDEO)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video: {TEST_VIDEO}")
    print(f"FPS: {fps}, Frames: {total_frames}")
    
    # Analyse: Jeden 5. Frame
    ball_positions = []
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_idx % 5 == 0:
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            detections = model.predict(image, threshold=0.3)
            
            for j in range(len(detections)):
                cls_name = detections.data.get('class_name', ['?'])[j] if 'class_name' in detections.data else '?'
                if cls_name == 'ball':
                    box = detections.xyxy[j]
                    cx = (box[0] + box[2]) / 2
                    cy = (box[1] + box[3]) / 2
                    ball_positions.append({
                        'frame': frame_idx,
                        'time': frame_idx / fps,
                        'x': float(cx / frame.shape[1]),
                        'y': float(cy / frame.shape[0]),
                        'conf': float(detections.confidence[j])
                    })
        
        frame_idx += 1
        if frame_idx % 100 == 0:
            print(f"  Frame {frame_idx}/{total_frames}...")
    
    cap.release()
    
    print(f"\nBall erkannt in {len(ball_positions)} von {total_frames // 5} analysierten Frames")
    
    # Ball-Positionen als JSON speichern
    with open('ball_track.json', 'w') as f:
        json.dump(ball_positions, f, indent=2)
    print("Ball-Track gespeichert als ball_track.json")
else:
    print(f"Kein Test-Video gefunden ({TEST_VIDEO}). Ueberspringe Video-Test.")
    print("Lade ein Video hoch und setze TEST_VIDEO auf den Pfad.")

## Schritt 7: Modell herunterladen

Lade `tt-detector.onnx` herunter und lege es in dein Projekt:

```
ttv-champions-web/
  public/
    models/
      tt-detector.onnx    <-- HIER
```

Das Modell wird dann automatisch von `video-ai-detector.js` geladen.

In [ ]:
# Modell herunterladen (Google Colab)
try:
    from google.colab import files
    files.download('tt-detector.onnx')
    print("Download gestartet!")
except ImportError:
    print("Nicht in Google Colab - lade die Datei manuell herunter:")
    print(f"  {os.path.abspath('tt-detector.onnx')}")

## Tipps fuer bessere Ergebnisse

### Mehr Daten = besser
- **300 Bilder**: Grundfunktion, viele Fehler
- **1000 Bilder**: Gute Genauigkeit
- **3000+ Bilder**: Sehr zuverlaessig

### Augmentation in Roboflow
- **Flip** (horizontal): Verdoppelt Datenmenge
- **Brightness** (+/- 25%): Verschiedene Beleuchtung
- **Blur** (0-2px): Simuliert Bewegungsunschaerfe
- **Noise** (bis 5%): Robustheit gegen Rauschen

### Schwierige Faelle
- Ball hinter Netz (teilweise verdeckt)
- Ball in der Luft (Bewegungsunschaerfe)
- Verschiedene Ballfarben (weiss, orange)
- Verschiedene Tischfarben (blau, gruen)
- Verschiedene Kamerawinkel

### Iteratives Training
1. Trainiere mit 300 Bildern
2. Teste auf neuen Videos
3. Finde Fehler (False Positives/Negatives)
4. Labele genau diese schwierigen Faelle
5. Trainiere erneut mit erweitertem Dataset
6. Wiederhole bis zufrieden